In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import pandas as pd
import csv
import scipy.io

%matplotlib inline

In [2]:
raw_data = open('train.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
X = np.array([np.array(xi) for xi in x])

In [24]:
X = pd.read_csv('train.csv')
X.iloc[:,[2,4,5,6,7,9,]]


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,22.0,1,0,7.2500
1,1,0,38.0,1,0,71.2833
2,3,0,26.0,0,0,7.9250
3,1,0,35.0,1,0,53.1000
4,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000
887,1,0,19.0,0,0,30.0000
888,3,0,NaN,1,2,23.4500
889,1,1,26.0,0,0,30.0000


In [33]:
def intialize(n_X,n_y,n_h1,n_h2):
    W1 = np.random.randn(n_h1,n_X)
    b1 = np.zeros((n_h1,1))
    W2 = np.random.randn(n_h2,n_h1)
    b2 = np.zeros((n_h2,1))
    W3 = np.random.randn(n_y,n_h2)
    b3 = np.zeros((n_y,1))
    #params = {"W1":W1, "b1":b1, "W2":W2, "b2":b2, "W1":W3, "b1":b3}
    return W1,b1,W2,b2,W3,b3

In [26]:
def relu (z):
    r = max (0, z)
    return r

def sigmoid(z):
    g = 1.0 / (1.0 + np.exp(-z))
    return g

In [28]:
def fwd(X,W1,b1,W2,b2,W3,b3,y):
    m = X.shape[1]
    Z1 = (np.dot(W1,X) + b1)  
    A1 = relu(Z1)

    Z2 = (np.dot(W2,A1) + b2)
    A2 = relu(Z2);

    Z3 = (np.dot(W3,A2) + b3)
    A3 = sigmoid(Z3);
    
    return A3,Z3,A2,Z2,A1,Z1

In [29]:
def bwd(X,y,A3,A1,A2,W3,W2,W1):

    m = X.shape[1]
    dZ3 = A3 - y;
    dW3 = 1/m * np.dot(dZ3,A2.T)
    db3 = 1/m * np.sum(dZ3,axis = 1, keepdims = True)
    dZ2 = np.dot(W3.T,dZ3) * (1 - np.exp(A2,2))
    dW2 = 1/m * np.dot(dZ2,A1.T)
    db2 = 1/m * np.sum(dZ2, axis = 1, keepdims = True)
    dZ1 = np.dot(W2.T,dZ2) * (1 - np.exp(A1,2))
    dW1 = 1/m * np.dot(dZ1,X.T)
    db1 = 1/m * np.sum(dZ1, axis = 1, keepdims = True)
                 
    return dW1,db1,dW2,db2,dW3,db3

In [31]:
def compute_cost(y,A3):
    
    m = y.shape[1]
    cost = -1/m * np.sum(y * np.log(A3)+(1-y) * np.log(1-A3),axis = 1,keepdims = True)

    return cost

In [32]:
def update(W1,b1,W2,b2,W3,b3,dW1,db1,dW2,db2,dW3,db3,alpha):

    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    return W1,b1,W2,b2,W3,b3

In [35]:
def nn_model(X, y, n_h1, n_h2, num_iterations = 10000, print_cost=False):
    
    np.random.seed(3)
    n_x = X.shape[0]
    n_y = y.shape[0]
    
    W1,b1,W2,b2,W3,b3 = intialize(n_X,n_y,n_h1,n_h2)

    for i in range(0, num_iterations):
        A3,Z3,A2,Z2,A1,Z1 = fwd(X,W1,b1,W2,b2,W3,b3,y)
        cost = compute_cost(y,A3)
        dW1,db1,dW2,db2,dW3,db3 = bwd(X,y,A3,A1,A2,W3,W2,W1)
        W1,b1,W2,b2,W3,b3 = update(W1,b1,W2,b2,W3,b3,dW1,db1,dW2,db2,dW3,db3,alpha)

        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return W1,b1,W2,b2,W3,b3

In [37]:
def predict(W1,b1,W2,b2,W3,b3,X,y):

    A3,Z3,A2,Z2,A1,Z1 = fwd(X,W1,b1,W2,b2,W3,b3,y)
    predictions = A3>0.5
    print(predictions)
    print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')
    
    return predictions